[View in Colaboratory](https://colab.research.google.com/github/mizunashi92/dlhub/blob/master/Sentiment.ipynb)

In [0]:
import sys
import re
!git clone https://github.com/openai/generating-reviews-discovering-sentiment.git
import numpy as np
import pandas as pd
from google.colab import files
!pip install reactionrnn

In [0]:
'''
Google Translate based on codes by mouuff at https://github.com/mouuff/mtranslate
'''
if (sys.version_info[0] < 3):
    import urllib2
    import urllib
    import HTMLParser
else:
    import html.parser
    import urllib.request
    import urllib.parse

agent = {'User-Agent':
"Mozilla/4.0 (\
compatible;\
MSIE 6.0;\
Windows NT 5.1;\
SV1;\
.NET CLR 1.1.4322;\
.NET CLR 2.0.50727;\
.NET CLR 3.0.04506.30\
)"}


def unescape(text):
    if (sys.version_info[0] < 3):
        parser = HTMLParser.HTMLParser()
    else:
        parser = html.parser.HTMLParser()
    return (parser.unescape(text))


def translate(to_translate, to_language="auto", from_language="auto"):
    """Returns the translation using google translate
    you must shortcut the language you define
    (French = fr, English = en, Spanish = es, etc...)
    if not defined it will detect it or use english by default
    Example:
    print(translate("salut tu vas bien?", "en"))
    hello you alright?
    """
    base_link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"
    if (sys.version_info[0] < 3):
        to_translate = urllib.quote_plus(to_translate)
        link = base_link % (to_language, from_language, to_translate)
        request = urllib2.Request(link, headers=agent)
        raw_data = urllib2.urlopen(request).read()
    else:
        to_translate = urllib.parse.quote(to_translate)
        link = base_link % (to_language, from_language, to_translate)
        request = urllib.request.Request(link, headers=agent)
        raw_data = urllib.request.urlopen(request).read()
    data = raw_data.decode("utf-8")
    expr = r'class="t0">(.*?)<'
    re_result = re.findall(expr, data)
    if (len(re_result) == 0):
        result = ""
    else:
        result = unescape(re_result[0])
    return (result)

In [16]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving datasource.csv to datasource.csv
User uploaded file "datasource.csv" with length 4100882 bytes


In [0]:
data = pd.read_csv("datasource.csv", sep='|')
data = data.drop_duplicates(subset='tweet_id', keep="last")

In [0]:
english = []
for i in range(len(data)):
    a = data['tweet_text'].iat[i].lower().replace('#indonesiatanpastres','').replace('\n',' ').replace('  ',' ')
    english.append(a)
    
english_tweet = []
for i in range(len(english)):
    b = translate(english[i])
    english_tweet.append(b)
    if i%50 == 0:
        print('The machine has pre-processed {n} data. There are {s} data left'.format(n=i, s=len(english)-i))

<h2>Working on the Sentiment Analysis Part</h2>

In [22]:
%cd generating-reviews-discovering-sentiment

/content/generating-reviews-discovering-sentiment/generating-reviews-discovering-sentiment


In [23]:
!pip install tqdm

In [0]:
from encoder import Model

model = Model()

In [25]:
'''
We will loop through the english tweets to get the reaction predictions and store them in text_features array
'''
text_features = model.transform(english_tweet)


627.226 seconds to transform 829 examples


In [0]:
a = text_features[:, 2388]

In [0]:
'''
We will re-combine the values with the initial comments in original language for easy readings.
'''
b = data.index.get_values() #collect the indexes of the original table
data2 = pd.DataFrame(data=a[0:], index=b[0:], columns=['Value']) #Saving the results as a new pd dataframe based on the index of original file stored on b
data3 = pd.DataFrame(data=np.array(english)[0:], index = b[0:], columns=['tweet_text']) #Fetch the original tweet texts as a new pandas dataframe



<h2>Working on the Reaction Analysis</h2>

In [0]:
!wget https://modeldepot.io/assets/uploads/models/models/e76620d2-41b0-4876-a4ba-d51388fe3444_reactionrnn_weights.hdf5
from reactionrnn import reactionrnn
reaction_weights='e76620d2-41b0-4876-a4ba-d51388fe3444_reactionrnn_weights.hdf5'
react = reactionrnn(reaction_weights)

In [0]:
'''
We will loop through the english tweets to get the reaction predictions and store them in ro array
'''
ro =[]
for i in range(len(english_tweet)):
  ve = react.predict(english_tweet[i])
  ro.append(ve)
  

In [0]:
data10 = pd.DataFrame(data=np.array(ro)[0:], index=b[0:], columns=['emo']) #Saving the results as a new pd dataframe based on the index of original file stored on b

<h2>Create the result table</h2>

In [0]:
result_table = pd.concat([data3, data2, data10], axis=1)

In [0]:
result_table.to_csv('result2.csv', sep ="|", encoding='utf-8')

In [0]:
files.download('result2.csv')

In [133]:
result_table

,tweet_text,Value,emo
5,rt @oriorio22: trainingnya gausah dibawa seriu...,-0.050398,"{'love': 0.8395, 'wow': 0.0814, 'haha': 0.0791..."
6,mau kuliah gratis di jerman? baca ini dulu! ht...,0.329331,"{'love': 0.5483, 'wow': 0.347, 'sad': 0.1048, ..."
7,ga mikir aneh2.. syukuri apa adanya.. hilangi...,0.016488,"{'love': 0.504, 'haha': 0.4437, 'wow': 0.0523,..."
8,rt @archtechnoyd: : 1. ketuhanan y.m.e 2. kema...,-0.048879,"{'love': 0.8326, 'haha': 0.0982, 'wow': 0.0691..."
13,rt @breflywesly18: : 1. ga perlu kerja mati-ma...,0.114726,"{'love': 0.5368, 'wow': 0.1819, 'angry': 0.158..."
14,nonton spongebob,-0.020496,"{'love': 0.728, 'haha': 0.2038, 'wow': 0.0681,..."
15,suka banget sama hestek twitter hari ini :d ha...,0.093560,"{'haha': 0.8436, 'love': 0.1443, 'wow': 0.0121..."
16,akan terwujud bila pemimpin &amp; rakyatnya d...,0.306276,"{'love': 0.8823, 'haha': 0.0707, 'wow': 0.047,..."
18,hidup bahagia saja.. susah lewati saja..,0.565432,"{'haha': 0.6492, 'love': 0.2389, 'wow': 0.1119..."
19,trainingnya gausah dibawa serius kali haha,0.067736,"{'haha': 0.6251, 'love': 0.2174, 'wow': 0.1575..."
